In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rcParams['font.family'] = "AppleGothic"

import json
with open('/Users/hyo-cheolahn/Documents/Projects/Book-Recommendation/data/scraping/all_book_data_ver2_combiined.json', "r") as file:
    data = json.load(file)

In [2]:
import pandas as pd

texts = []
for el in data:
    t1 = [x.split(':')[-1].strip() for x in el['description'].split('\n')] # 여러개의 라인으로 된 데이터 정리
    texts.append(' '.join(t1))
df = pd.DataFrame()
df['texts'] = texts
df.head()


,texts
0,행복을 찾는 방법이 아니라불행에 대한 수비력을 길러주는58가지 인생 이야기《1cm ...
1,"좋은 감각을 지니려면,디자인을 잘하려면,더 나은 브랜드를 만들려면 어떻게 해야 하는..."
2,“코난밖에 모르던 내가 일본어로 먹고살아요‘그저 좋아하다가 잘하게 된 센님의 일본어...
3,"당신의 행복을 찾아 주고 싶다이렇게 행복해도 되나 싶을 정도로이런 하루가, 이런 인..."
4,"디 에센셜 한강한강 작가의 장편소설, 단편소설, 시, 산문을 한 권으로 만난다!작가..."


In [3]:
import pandas as pd

texts = []
for el in data:
    t1 = [x.split(':')[-1].strip() for x in el['key_sentences'].split('\n')] # 여러개의 라인으로 된 데이터 정리
    texts.append(' '.join(t1))
df['texts_ks'] = texts
df.head()


,texts,texts_ks
0,행복을 찾는 방법이 아니라불행에 대한 수비력을 길러주는58가지 인생 이야기《1cm ...,"그냥 지금처럼 살아라. 그렇게 살되 어떤 감정조차 책임질 수 없을 만큼 힘든 날, ..."
1,"좋은 감각을 지니려면,디자인을 잘하려면,더 나은 브랜드를 만들려면 어떻게 해야 하는...","여러 회사와 각종 분야, 다양한 프로젝트를 넘나들며 일을 하면서도, 늘 저의 중심을..."
2,“코난밖에 모르던 내가 일본어로 먹고살아요‘그저 좋아하다가 잘하게 된 센님의 일본어...,하지만 오로지 애니를 보기만 했을 뿐 다른 노력은 전혀 하지 않았는데 이만큼이나 들...
3,"당신의 행복을 찾아 주고 싶다이렇게 행복해도 되나 싶을 정도로이런 하루가, 이런 인...","오늘도 괜찮다. 덜 걱정해도 될 일이고, 그만 불안해도 되는 날이다. 다 지나간 일..."
4,"디 에센셜 한강한강 작가의 장편소설, 단편소설, 시, 산문을 한 권으로 만난다!작가...","그때에는 그녀에게 말言이 있었으므로, 감정들은 더 분명하고 강했을 것이다.그러나 지..."


In [4]:

df_combined = df['texts'].str.cat(df['texts_ks'], sep=' | ')
df_combined.head()

0    행복을 찾는 방법이 아니라불행에 대한 수비력을 길러주는58가지 인생 이야기《1cm ...
1    좋은 감각을 지니려면,디자인을 잘하려면,더 나은 브랜드를 만들려면 어떻게 해야 하는...
2    “코난밖에 모르던 내가 일본어로 먹고살아요‘그저 좋아하다가 잘하게 된 센님의 일본어...
3    당신의 행복을 찾아 주고 싶다이렇게 행복해도 되나 싶을 정도로이런 하루가, 이런 인...
4    디 에센셜 한강한강 작가의 장편소설, 단편소설, 시, 산문을 한 권으로 만난다!작가...
Name: texts, dtype: object

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# we add this to remove stopwords
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

model = BERTopic(
    vectorizer_model=vectorizer_model,
    language='english', calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(df_combined)

In [2]:
from bertopic import BERTopic
print("BERTopic imported successfully!")

/opt/anaconda3/envs/sesac/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 